This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [1]:
import pandas as pd
import numpy as np
import os
import random
random.seed(1)
import re
import time

# Data processing.
import constants # constants.py
import dataset # dataset.py
import torch
from torchtext.legacy import data 

# Model.
import models # models.py
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertModel, DistilBertTokenizer

# Training.
import training # training.py
from sklearn.model_selection import KFold
import utils # utils.py

# Visualization.
import matplotlib.pyplot as plt

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Load a BERT tokenizer

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained(constants.WEIGHTS_NAME)

# Read the data

In [3]:
data_df = dataset.get_multiple_datasets([1,2,3], 'Creativity_Combined', shuffle=True)

In [4]:
# For prototype purposes:
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
train_df = data_df[:1000]
test_df = data_df[1000:] # roughly 190 test examples set aside

# write them to CSV files
train_df.to_csv('ktrain.csv', index=False, header=False)
test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [5]:
INIT_TOKEN_IDX = tokenizer.cls_token_id
EOS_TOKEN_IDX = tokenizer.sep_token_id
PAD_TOKEN_IDX = tokenizer.pad_token_id
UNK_TOKEN_IDX = tokenizer.unk_token_id

# BERT input can be at most 512 words
MAX_INPUT_LENGTH = tokenizer.max_model_input_sizes[constants.WEIGHTS_NAME]

# Apply tokenization and some preprocessing steps to the input sentence.
# Namely, this trims examples down to MAX_INPUT_LENGTH. (There is a -2 
# since the [CLS] and [SEP] tokens will be added)
def tokenize_and_cut(sentence):
  sentence = sentence.replace('/', '') # remove slashes
  tokens = tokenizer.tokenize(sentence) 
  tokens = tokens[:MAX_INPUT_LENGTH-2]
  return tokens

# text_fields defines preprocessing and handling of the text of an example.
text_fields = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = INIT_TOKEN_IDX, # add [CLS] token
                  eos_token = EOS_TOKEN_IDX, # add [SEP] token
                  pad_token = PAD_TOKEN_IDX,
                  unk_token = UNK_TOKEN_IDX)

# label_fields defines how to handle the label of an example.
# for regression, we do not need to build a vocabulary.
label_fields = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)
all_fields = [('text', text_fields), ('label', label_fields)] # must match order of cols in csv

train_dataset, test_dataset = data.TabularDataset.splits(
  path='', # path='' because the csvs are in the same directory
  train='ktrain.csv', test='ktest.csv', format='csv',
  fields=all_fields  
)

In [6]:
print(data_df.head(1))

                                                  text  label
233  A weight to track repetitions and keep track o...    3.2


In [7]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


## Define parameters

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)

params = {
    'dropout': 0.2
    'batch_size': 8
    'n_epochs': 3
    'lr': 5e-05
}

valid_corrs = training.launch_experiment(1, train_exs_arr, constants.N_SPLITS, params, save_weights=True)
print('validation correlations: {}'.format(valid_corrs))

training on fold 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 9.623 | Train Corr: 0.41
	 Val. Loss: 7.062 |  Val. Corr: 0.10
updating saved weights of best model
Epoch: 01 | Epoch Time: 0m 47s
	 Train Loss: 3.264 | Train Corr: 0.56
	 Val. Loss: 5.380 |  Val. Corr: 0.25
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 47s
	 Train Loss: 2.863 | Train Corr: 0.67
	 Val. Loss: 3.849 |  Val. Corr: 0.21
training on fold 1


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 9.472 | Train Corr: 0.40
	 Val. Loss: 6.539 |  Val. Corr: 0.08
Epoch: 01 | Epoch Time: 0m 48s
	 Train Loss: 3.438 | Train Corr: 0.54
	 Val. Loss: 6.170 |  Val. Corr: 0.26
Epoch: 02 | Epoch Time: 0m 47s
	 Train Loss: 3.256 | Train Corr: 0.56
	 Val. Loss: 4.580 |  Val. Corr: 0.43
training on fold 2


...

Epoch: 00 | Epoch Time: 0m 47s
	 Train Loss: 8.399 | Train Corr: 0.49
	 Val. Loss: 3.940 |  Val. Corr: 0.17
Epoch: 01 | Epoch Time: 0m 47s
	 Train Loss: 3.550 | Train Corr: 0.62
	 Val. Loss: 4.380 |  Val. Corr: 0.49
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 46s
	 Train Loss: 2.361 | Train Corr: 0.70
	 Val. Loss: 1.811 |  Val. Corr: 0.61
training on fold 3


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 45s
	 Train Loss: 9.450 | Train Corr: 0.43
	 Val. Loss: 7.454 |  Val. Corr: 0.20
Epoch: 01 | Epoch Time: 0m 44s
	 Train Loss: 3.473 | Train Corr: 0.58
	 Val. Loss: 4.119 |  Val. Corr: 0.42
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 44s
	 Train Loss: 2.922 | Train Corr: 0.64
	 Val. Loss: 1.758 |  Val. Corr: 0.65
training on fold 4


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 45s
	 Train Loss: 8.628 | Train Corr: 0.43
	 Val. Loss: 4.942 |  Val. Corr: 0.19
Epoch: 01 | Epoch Time: 0m 45s
	 Train Loss: 3.044 | Train Corr: 0.65
	 Val. Loss: 4.628 |  Val. Corr: 0.23
Epoch: 02 | Epoch Time: 0m 45s
	 Train Loss: 2.189 | Train Corr: 0.76
	 Val. Loss: 3.328 |  Val. Corr: 0.29
best validation loss: 1.758255541920662


# Test the trained model on held-out dataset.

In [11]:
# Get a test iterator
test_iterator = training.get_iterator(test_dataset, params['batch_size'], device)

In [12]:
# load the best model saved
bert = DistilBertModel.from_pretrained(constants.WEIGHTS_NAME)
model = models.BERTLinear(bert, constants.OUTPUT_DIM, params['dropout'])
model.load_state_dict(torch.load("best_valid_loss.pt"))
model.to(device)
model.eval()
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predictions: tensor([3.7420, 3.1371, 2.9669, 4.6738, 4.3069, 4.3898, 3.0036, 3.5888],
       device='cuda:0')
true labels: tensor([4.1750, 2.6750, 3.3500, 4.3250, 4.1750, 4.5750, 3.2500, 4.9500],
       device='cuda:0')
predictions: tensor([3.4298, 4.1198, 3.7915, 3.9154, 3.7454, 2.9854, 4.5348, 3.8341],
       device='cuda:0')
true labels: tensor([4.1250, 4.3500, 4.7000, 4.3000, 5.2750, 2.5500, 5.4250, 3.2750],
       device='cuda:0')
predictions: tensor([3.9121, 2.8993, 3.8251, 4.6524, 3.7348, 4.3551, 3.6334, 3.5260],
       device='cuda:0')
true labels: tensor([5.2750, 2.7750, 4.3750, 4.7750, 4.3750, 3.5500, 4.0500, 3.9250],
       device='cuda:0')
predictions: tensor([3.3758, 3.6365, 2.9481, 3.0048, 4.2327, 4.4699, 3.6686, 3.5043],
       device='cuda:0')
true labels: tensor([4.6000, 3.9250, 2.8500, 3.3000, 4.8000, 3.3250, 5.0500, 3.7250],
       device='cuda:0')
predictions: tensor([3.5478, 4.4148, 3.4240, 3.7057, 3.6455, 4.6826, 4.1336, 3.1818],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."